In [1]:
# AutoDL官方学术资源加速
import subprocess
import os

result = subprocess.run('bash -c "source /etc/network_turbo && env | grep proxy"', shell=True, capture_output=True, text=True)
output = result.stdout
for line in output.splitlines():
    if '=' in line:
        var, value = line.split('=', 1)
        os.environ[var] = value

In [2]:
import asyncio
import time
from openai import AsyncOpenAI  # 注意这里导入AsyncOpenAI
from typing import List, Dict
from tqdm import tqdm
import json

# config.py
from dotenv import load_dotenv # type: ignore

# 加载 .env 文件
load_dotenv()

# 读取
ZetaTechs_api_key = os.getenv('ZETATECHS_API_KEY')
ZetaTechs_api_base = os.getenv('ZETATECHS_API_BASE')

In [3]:
# class AsyncAPIClient:
#     def __init__(self, max_concurrency=100):
#         self.client = AsyncOpenAI(api_key=ZetaTechs_api_key, base_url=ZetaTechs_api_base)
#         self.semaphore = asyncio.Semaphore(max_concurrency)
#         self.results = []  # 存储所有结果
        
#     async def generate_response(self, prompt: str) -> str:
#         try:
#             async with self.semaphore:
#                 response = await self.client.chat.completions.create(
#                     model="gpt-4o-mini",
#                     messages=[
#                         {"role": "system", "content": """你是一个专业的系统提示词工程师。你的任务是根据给定的对话内容，生成恰当的系统提示词。

# 请注意以下要求：
# 1. 系统提示词应该定义AI助手的角色、行为准则和专业领域
# 2. 提示词应该包含AI助手的性格特征和交互风格
# 3. 明确指出AI助手在回答时应遵循的原则和限制
# 4. 确保提示词简洁专业，通常以"你是..."或"作为..."等开头
# 5. 避免生成看起来像用户指令的提示词

# 示例格式：
# "你是一个专业的[领域]专家，擅长[特定能力]。在回答问题时，你应该[行为准则]，并始终保持[性格特征]。"

# 请分析用户提供的对话内容，生成一个符合以上要求的系统提示词。直接返回提示词内容，不需要任何解释和额外的内容。"""},
#                         {"role": "user", "content": prompt}
#                     ]
#                 )
#                 return response.choices[0].message.content
#         except Exception as e:
#             print(f"请求失败: {e}")
#             return f"Error: {str(e)}"

#     async def process_batch(self, prompts: List[str], batch_size: int):
#         """分批处理提示词"""
#         for i in range(0, len(prompts), batch_size):
#             batch = prompts[i:i + batch_size]
#             print(f"\n处理批次 {i//batch_size + 1}, 项目 {i}-{i+len(batch)}")
            
#             # 处理当前批次
#             batch_results = await self.batch_generate(batch)
            
#             # 保存当前批次结果
#             self.save_results(batch_results, i//batch_size)
            
#             # 将结果添加到总结果列表
#             self.results.extend(batch_results)

#     async def batch_generate(self, prompts: List[str]) -> List[str]:
#         tasks = [self.generate_response(prompt) for prompt in prompts]
#         results = await asyncio.gather(*tasks)
#         return results

#     def save_results(self, batch_results: List[str], batch_num: int):
#         """保存批次结果"""
#         # 保存当前批次
#         batch_file = f"base_stage_data_raw/system_prompts_batch_{batch_num}.json"
#         with open(batch_file, "w", encoding="utf-8") as f:
#             json.dump(batch_results, f, ensure_ascii=False, indent=4)
        
#         # 更新总结果文件
#         full_file = "base_stage_data_raw/system_prompts_all.json"
#         with open(full_file, "w", encoding="utf-8") as f:
#             json.dump(self.results, f, ensure_ascii=False, indent=4)
        
#         print(f"已保存批次 {batch_num} 的结果，共 {len(batch_results)} 条")

In [4]:
class AsyncAPIClient:
    def __init__(self, max_concurrency=100):
        self.client = AsyncOpenAI(api_key=ZetaTechs_api_key, base_url=ZetaTechs_api_base)
        self.semaphore = asyncio.Semaphore(max_concurrency)
        self.results = []
        
    async def generate_response(self, prompt: str, max_retries=3, retry_delay=2) -> str:
        """发送单个请求到OpenAI API，包含重试机制"""
        retries = 0
        while retries <= max_retries:
            try:
                async with self.semaphore:
                    response = await self.client.chat.completions.create(
                        model="gpt-4o-mini",
                        messages=[
                            {"role": "system", "content": """你是一个专业的系统提示词工程师。你的任务是根据给定的对话内容，生成恰当的系统提示词。

    请注意以下要求：
    1. 系统提示词应该定义AI助手的角色、行为准则和专业领域
    2. 提示词应该包含AI助手的性格特征和交互风格
    3. 明确指出AI助手在回答时应遵循的原则和限制
    4. 确保提示词简洁专业，通常以"你是..."或"作为..."等开头
    5. 避免生成看起来像用户指令的提示词

    示例格式：
    "你是一个专业的[领域]专家，擅长[特定能力]。在回答问题时，你应该[行为准则]，并始终保持[性格特征]。"

    请分析用户提供的对话内容，生成一个符合以上要求的系统提示词。请直接返回提示词内容，不需要任何解释和额外的内容。"""},
                            {"role": "user", "content": prompt}
                        ]
                    )
                    return response.choices[0].message.content
                    
            except Exception as e:
                retries += 1
                if retries <= max_retries:
                    error_msg = str(e)
                    if "Connection error" in error_msg or "Request timed out" in error_msg:
                        print(f"请求失败 (尝试 {retries}/{max_retries}): {error_msg}")
                        # 指数退避重试延迟
                        await asyncio.sleep(retry_delay * (2 ** (retries - 1)))
                        continue
                
                print(f"请求最终失败: {e}")
                return f"Error: {str(e)}"

    async def process_batch(self, prompts: List[str], batch_size: int):
        """分批处理提示词"""
        for i in range(0, len(prompts), batch_size):
            batch = prompts[i:i + batch_size]
            print(f"\n处理批次 {i//batch_size + 1}, 项目 {i}-{i+len(batch)}")
            
            # 处理当前批次
            batch_results = await self.batch_generate(batch)
            
            # 过滤出成功的结果和失败的项目
            successful_results = []
            failed_indices = []
            
            for idx, result in enumerate(batch_results):
                if result.startswith("Error:"):
                    failed_indices.append(i + idx)
                else:
                    successful_results.append(result)
            
            # 保存当前批次结果
            self.save_results(successful_results, i//batch_size)
            
            # 记录失败的项目
            if failed_indices:
                self.save_failed_items(failed_indices, i//batch_size)
            
            # 将成功的结果添加到总结果列表
            self.results.extend(successful_results)

    async def batch_generate(self, prompts: List[str]) -> List[str]:
        """批量处理多个提示词"""
        tasks = [self.generate_response(prompt) for prompt in prompts]
        results = await asyncio.gather(*tasks)
        return results

    def save_results(self, batch_results: List[str], batch_num: int):
        """保存批次结果"""
        # 保存当前批次
        batch_file = f"stage1/system_prompts_gpt/train_system_prompt_batch_{batch_num}.json"
        with open(batch_file, "w", encoding="utf-8") as f:
            json.dump(batch_results, f, ensure_ascii=False, indent=4)
        
        # 更新总结果文件
        full_file = "stage1/system_prompts_gpt/train_system_prompt.json"
        with open(full_file, "w", encoding="utf-8") as f:
            json.dump(self.results, f, ensure_ascii=False, indent=4)
        
        print(f"已保存批次 {batch_num} 的结果，共 {len(batch_results)} 条")

    def save_failed_items(self, failed_indices: List[int], batch_num: int):
        """保存失败的项目信息"""
        failed_file = f"stage1/system_prompts_gpt/failed_train_system_prompt_batch_{batch_num}.json"
        with open(failed_file, "w", encoding="utf-8") as f:
            json.dump({"failed_indices": failed_indices}, f, ensure_ascii=False, indent=4)
        print(f"批次 {batch_num} 中有 {len(failed_indices)} 个项目失败，已记录")

In [5]:
# 使用示例
async def main():
    # 设置并发数和批次大小
    max_concurrency = 50
    batch_size = 50
    
    # 初始化客户端
    client = AsyncAPIClient(max_concurrency)
    
    # 读取需要处理的数据
    with open('stage1/data_raw/train.json', 'r', encoding='utf-8') as f:
        train_data = json.load(f)
    
    # 提取text字段
    prompts = [item['text'] for item in train_data if 'text' in item]
    
    # 处理所有数据
    await client.process_batch(prompts[:30000+1000], batch_size)  # 这里示例只处理train_data的前31000条
    
    print("所有数据处理完成！")

In [ ]:
# 在Jupyter中运行
if __name__ == "__main__":
    await main()